In [6]:
from google_play_scraper import app, reviews, Sort, reviews_all

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import datetime as dict
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from wordcloud import WordCloud

import nltk
nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lucky\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
scrapreview = reviews_all(
    'com.byu.id',
    lang="id",
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000
)

In [8]:
# Menyimpan ulaasan dalam file CSV
import csv

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])
    for review in scrapreview:
        writer.writerow([review['content']])

In [10]:
# Loading Dataset
app_reviews_df = pd.DataFrame(scrapreview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv("ulasan_aplikasi.csv", index=False)

# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview)

jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

In [11]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,36014f67-2f37-4034-8cfc-a39f76f01ed5,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sinyal bagus tapi buat buka apk nya lemot bang...,1,3,1.63.1,2025-10-15 16:06:42,Hallo Kak. Maaf banget udah bikin gak nyaman. ...,2025-10-15 16:08:06,1.63.1
1,269f6fd9-615f-4ed6-9dd8-fd9411a82caf,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"untuk developer ,aplikasinya lemot dikit dikit...",1,2,1.63.0,2025-10-14 21:42:53,"Hai Kak, maaf ya udah bikin ga nyaman :( Terka...",2025-10-14 21:43:41,1.63.0
2,ce1e598d-e54f-4a8d-9d1c-5fea795a39cc,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"udh hampir 1 tahun pake by u, aplikasi nya ber...",1,14,1.63.0,2025-10-12 20:18:16,"Hai Kak, maaf ya udah bikin ga nyaman :( Terka...",2025-10-12 20:19:00,1.63.0
3,695ba5b6-4df5-467d-80f5-afa1a7172c28,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"KECEWA, paket ""Yang Bikin Makin Aman Jaya"" tib...",1,13,1.63.0,2025-10-13 08:38:39,"Hai Kak, maaf ya udah bikin ga nyaman :( Terka...",2025-10-13 10:40:59,1.63.0
4,47343457-0877-41d5-97ba-39e07d4882e6,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kenapa sih tiap buka apk nya tiba² keluar send...,3,10,1.63.0,2025-10-11 21:00:00,"Hai Kak, maaf nih terkait kendala jaringan, bi...",2025-10-11 21:03:03,1.63.0


In [12]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153000 entries, 0 to 152999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              153000 non-null  object        
 1   userName              153000 non-null  object        
 2   userImage             153000 non-null  object        
 3   content               152999 non-null  object        
 4   score                 153000 non-null  int64         
 5   thumbsUpCount         153000 non-null  int64         
 6   reviewCreatedVersion  130007 non-null  object        
 7   at                    153000 non-null  datetime64[ns]
 8   replyContent          141957 non-null  object        
 9   repliedAt             141957 non-null  datetime64[ns]
 10  appVersion            130007 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 12.8+ MB


In [13]:
clean_df = app_reviews_df.dropna()

In [14]:
clean_df = clean_df.drop_duplicates()

jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

In [15]:
def cleaningText(text):
    text = re.sub(r"@[A-Za-z0-9]+", '', text) # Delete mention
    text = re.sub(r"#[A-Za-z0-9]+", '', text) # Delete hashtag
    text = re.sub(r"RT[\s]", "", text) # Delete RT
    text = re.sub(r"http\S+", "", text) # Delete Link
    text = re.sub(r"[0-9]+", "", text) # Delete Numbers
    text = re.sub(r"[^\w\s]", "", text) # Delete character except huruf and number

    text = text.replace("\n", "")
    text = text.translate(str.maketrans("", "", string.punctuation)) # Menghapus semua tanda baca
    text = text.strip(" ")
    
    return text

def casefoldingText(text):
    text = text.lower()
    return text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(text):
    listStopwords = set(stopwords.words("indonesian"))
    listStopwords1 = set(stopwords.words("english"))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered

    return text

def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    words = text.split()

    stemmed_words = [stemmer.stem(word) for word in words]

    stemmed_text = " ".join(stemmed_words)

    return stemmed_text

def toSentence(list_words):
    sentence = " ".join(word for word in list_words)
    return sentence


In [16]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = " ".join(fixed_words)
    return fixed_text

In [17]:
# membersikan text dan menyimpannya di kolom "text_clean"
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi hurf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata kata slang dengan kata kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tekenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpanya di 'text_akhir'
clean_df["text_akhir"] = clean_df['text_stopword'].apply(toSentence)

In [20]:
import csv
import requests
from io import StringIO
 
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = {}
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")
 
# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = {}
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [ ]:
# Fungsi untuk menentukan polaritas sentimaen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    score = 0

    for word in text:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]

    for word in text:
        if(word in lexicon_negative):
            score = score + lexicon_negative[word]

    polarity = ""

    if(score >= 0):
        polarity = "positive"
    elif (score < 0):
        polarity = "negative"

    return score, polarity


In [24]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    63165
negative    56953
Name: count, dtype: int64
